In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import logging 
logging.basicConfig(level=logging.INFO)
import atlite

In [2]:
filename="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
ds = xr.open_dataset(filename)

In [7]:
ds

<xarray.Dataset> Size: 17GB
Dimensions:           (x: 189, y: 157, time: 8784)
Coordinates:
  * x                 (x) float64 2kB -12.0 -11.75 -11.5 ... 34.5 34.75 35.0
  * y                 (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.5 71.75 72.0
  * time              (time) datetime64[ns] 70kB 2016-01-01 ... 2016-12-31T23...
    lon               (x) float64 2kB ...
    lat               (y) float64 1kB ...
Data variables: (12/13)
    height            (y, x) float32 119kB ...
    wnd100m           (time, y, x) float32 1GB ...
    wnd_azimuth       (time, y, x) float32 1GB ...
    roughness         (time, y, x) float32 1GB ...
    influx_toa        (time, y, x) float32 1GB ...
    influx_direct     (time, y, x) float32 1GB ...
    ...                ...
    albedo            (time, y, x) float32 1GB ...
    solar_altitude    (time, y, x) float64 2GB ...
    solar_azimuth     (time, y, x) float64 2GB ...
    temperature       (time, y, x) float64 2GB ...
    soil temperature  (time, y, x) float64 2GB ...
    runoff            (time, y, x) float32 1GB ...
Attributes:
    module:             era5
    prepared_features:  ['influx', 'temperature', 'runoff', 'height', 'wind']
    chunksize_time:     100
    Conventions:        CF-1.6
    history:            2023-02-20 09:11:45 GMT by grib_to_netcdf-2.25.1: /op...

In [5]:
#cutout_path = "europe-2008-06-week-era5_new_wnd10_shear.nc"
cutout_path ="/groups/EXTREMES/cutouts/europe-2016-era5.nc"
 
# read in cutout instead of creating it
cutout = atlite.Cutout(path=cutout_path)

In [11]:
from __future__ import annotations

import datetime as dt
import logging
from collections import namedtuple
from operator import itemgetter
from pathlib import Path
from typing import TYPE_CHECKING

import geopandas as gpd
import numpy as np
import pandas as pd
import xarray as xr
from dask import compute, delayed
from dask.array import absolute, arccos, cos, maximum, mod, radians, sin, sqrt
from dask.diagnostics import ProgressBar
from numpy import pi
from scipy.sparse import csr_matrix

from atlite.aggregate import aggregate_matrix
from atlite.gis import spdiag
from atlite.pv.irradiation import TiltedIrradiation
from atlite.pv.orientation import SurfaceOrientation, get_orientation
from atlite.pv.solar_panel_model import SolarPanelModel
from atlite.pv.solar_position import SolarPosition
from atlite.resource import (
    get_cspinstallationconfig,
    get_solarpanelconfig,
    get_windturbineconfig,
    windturbine_smooth,
)

In [9]:
orientation='latitude_optimal'

In [15]:
cutout.pv('CSi.yaml', orientation) #trying to get pv with 1 year of era5 data

INFO:atlite.convert:Convert and aggregate 'pv'.


<xarray.DataArray 'specific generation' (y: 157, x: 189)> Size: 237kB
array([[1665.117087  , 1670.82985761, 1675.62453298, ..., 1733.68983396,
        1724.15118419, 1713.5150573 ],
       [1654.5333218 , 1660.26905172, 1667.04610104, ..., 1732.17706531,
        1724.57313397, 1720.71018939],
       [1646.86677728, 1653.54548917, 1660.31920306, ..., 1732.02486402,
        1725.92714883, 1722.08074039],
       ...,
       [ 668.541382  ,  663.58420748,  660.24277091, ...,  717.41877686,
         716.21786177,  715.0961002 ],
       [ 672.07585724,  666.2621657 ,  661.25752118, ...,  705.85857593,
         704.28351569,  702.56793633],
       [ 673.47312207,  667.58092958,  662.66832355, ...,  691.90428341,
         690.56089424,  688.89504447]])
Coordinates:
  * y        (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.25 71.5 71.75 72.0
    lat      (y) float64 1kB 33.0 33.25 33.5 33.75 ... 71.25 71.5 71.75 72.0
  * x        (x) float64 2kB -12.0 -11.75 -11.5 -11.25 ... 34.25 34.5 34.75 35.0
    lon      (x) float64 2kB -12.0 -11.75 -11.5 -11.25 ... 34.25 34.5 34.75 35.0
Attributes:
    units:    kWh/kWp